In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
# from transformers.generation import GenerationConfig


/home/user/anaconda3/envs/torch2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_dir = "./Qwen-14B-8bit-hf"

In [3]:
# !cp model_configs/* Qwen-14B-8bit-hf
# !mv Qwen-14B-8bit-hf/gptq_model-8bit-128g.bin Qwen-14B-8bit-hf/pytorch_model.bin

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True, use_fast=True)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True,
).eval()

In [6]:
prompt="""###
Article: (CNN Student News) -- November 9, 2012 . Download PDF maps related to today's show: . Greece . Guatemala . Japan . Michigan; Utah . Click here to access the transcript of today's CNN Student News program. Please note that there may be a delay between the time when the video is available and when the transcript is published.

Summarize the above article in 3 sentences.
The daily transcript is a written version of each day's CNN Student News program . Use this transcript to help students with reading comprehension and vocabulary . Use the weekly Newsquiz to test your knowledge of stories you saw on CNN Student News .

###
Article: KATHMANDU, Nepal (CNN) -- Two people were killed and about a dozen others were injured when a bomb exploded in a Catholic church in Kathmandu on Saturday morning, police said. The damage inside the church in Kathmandu following Saturday's bomb blast. The explosion in the Nepalese capital killed a 15-year-old girl and a 30-year-old woman. "The bomb exploded inside the church when the explosion happened," senior police officer Kedar Man Singh Bhandari told CNN over the phone. About 100 people were in the church when the bomb exploded, police said. Manish Amatya, who was injured, said the blast interrupted their prayers. "There was a loud explosion while we were praying and all of us ran out screaming," he said. Investigations are under way to determine who planted the bomb, which damaged the church. CNN's Manesh Shrestha contributed to this report.

Summarize the above article in 3 sentences.
Explosion in Nepalese capital killed 15-year-old girl, 30-year-old woman . 100 people were in the church when the bomb exploded . Investigations are under way to determine who planted the bomb .

###
Article: NEW DELHI, India (CNN) -- At least 441 people have died in floods in India from this season's monsoon rains, federal authorities said in their latest report. An Indian child plays in a flooded street in Mumbai earlier this month. Flooding has affected more than 1.5 million people in parts of India, said the disaster management division of the federal home ministry. The country's main weather office has warned of more heavy rain in western and central parts of India. Monsoon rains sweep across the subcontinent from June till September. Though they bring much-needed relief to often-parched farmlands, they also leave a trail of landslides, home collapses and floods that can kill. In neighboring Pakistan, torrential monsoon rains left more than three dozen people dead and broke a 32-year record over the weekend. CNN's Harmeet Shah Singh contributed to this report.

Summarize the above article in 3 sentences."""
"""
7 die as bus carrying 40 passengers sinks in overflowing canal in eastern India . 7-year-old girl and her mother among the dead . Bus driver ignored warnings from his passengers about flooding in canal .

###
Article: (CNN)Each day, CNN producers select a user-submitted photo to be our Travel Photo of the Day. Click through the gallery above to see stunning shots from around the world, and be sure to come back every day for a new image. Have a gorgeous travel photo of your own to share? Submit it for the gallery at CNN iReport!

Summarize the above article in 3 sentences.
See more iReport galleries: Glorious Ireland, beautiful beaches . Follow us on Twitter @cnnireport and @CNNTravel .

###
Article: NEW YORK (CNN) -- A nude photograph of pop singer Madonna was sold for $37,500 Thursday afternoon at a Christie's Art House auction. Christie's auctioned this nude photo of Madonna (partially shown) taken by Lee Friedlander for $37,500. The photo, originally expected to go for between $10,000 and $15,000, was purchased for more than double its original estimated selling price, a Christie's spokesperson confirmed. The 13-inch by 8 5/8-inch framed photograph was purchased by an anonymous bidder over the phone. The full frontal photograph was one of several taken by American photographer Lee Friedlander in 1979. Madonna, then a cash-strapped student, received $25 for the entire photo shoot. Most of the pictures from the shoot were ultimately featured in Playboy magazine in 1985.

Summarize the above article in 3 sentences.
Nude photograph of Madonna taken when she was student in 1979 . Lee Friedlander pic sold by Christie's for $37,500 . Anonymous bidder made purchase over the phone .

###
Article: More than 8.7million flyers were delivered by the company in the year following its privatisation . Royal Mail has stepped up the amount of junk mail it stuffs through our doors since it was sold off by the Government, new figures show. More than 8.7million flyers were delivered by the company in the year following its privatisation – sparking fears that it risks becoming a ‘leaflet distributor’. In the 12 months after privatisation, postmen pushed almost 50,000 more leaflets and advertising letters through our doors every single day, compared to the previous year. It means almost 3.2bn items of ‘unaddressed’ junk mail were delivered in just one year – almost double the 1.7bn delivered annually five years ago. These figures do not even include the vast quantities of ‘addressed’ junk mail also delivered, which have the names of occupants on the envelope. The rise in unsolicited mail has proved lucrative for Royal Mail, which is struggling from increased competition in the parcels market and a fall in the number of ordinary letters being posted. Revenue from ‘marketing mail’ – addressed and unaddressed – has soared to £1.1bn a year, earning them £3million a day. Critics last night called for Royal Mail to reduce the amount of ‘nuisance’ junk mail it delivers or risk becoming a ‘leaflet distributor’. Royal Mail admitted junk Mail had become an ‘important component’ of its business, and insisted many households found advertising flyers ‘useful’. But much of unwanted junk mail is never opened and ends up being thrown straight into the bin – causing an environmental headache. While some households recycle takeaway menus and pamphlets, many tonnes of junk mail end up in landfill – costing councils tens of millions of pounds to dispose of. A spokesman for the Local Government Association said: ‘Junk mail is a blight on the nation’s doormats. ‘Not only is it a nuisance to pick up and throw away, it adds thousands of tonnes to the waste councils have to pick up creating additional costs for tax payers. ‘Royal Mail and its direct marketing clients have to do far more to reduce the torrent of junk mail.’ Robert Rijkhoff, of the Stop Junk Mail campaign, said marketing mail was ‘the worst form of advertising’ because it was foisted upon households without their consent. He said: ‘Royal Mail needs to make money but, if they deliver more and more junk mail as a source of income, in the long term I am not sure it is a sustainable business. ‘Royal Mail is delivering less and less letters. They are risking that, in time, they will become a leaflet distributor.’ Critics claim that some junk mail contain scams sent by fraudsters from overseas. Critics said the company was running the risk of being labelled a 'leaflet distributor' with fewer letters being posted every year . In 2011, the National Fraud Authority claimed the Royal Mail logo was seen by some victims as a stamp of legitimacy, making the scammers appear more trustworthy. At the time, Mike Haley, director of the National Fraud Authority, Royal Mail

Summarize the above article in 3 sentences."""

"\n7 die as bus carrying 40 passengers sinks in overflowing canal in eastern India . 7-year-old girl and her mother among the dead . Bus driver ignored warnings from his passengers about flooding in canal .\n\n###\nArticle: (CNN)Each day, CNN producers select a user-submitted photo to be our Travel Photo of the Day. Click through the gallery above to see stunning shots from around the world, and be sure to come back every day for a new image. Have a gorgeous travel photo of your own to share? Submit it for the gallery at CNN iReport!\n\nSummarize the above article in 3 sentences.\nSee more iReport galleries: Glorious Ireland, beautiful beaches . Follow us on Twitter @cnnireport and @CNNTravel .\n\n###\nArticle: NEW YORK (CNN) -- A nude photograph of pop singer Madonna was sold for $37,500 Thursday afternoon at a Christie's Art House auction. Christie's auctioned this nude photo of Madonna (partially shown) taken by Lee Friedlander for $37,500. The photo, originally expected to go for b

In [10]:
# prompt = """Warning: please make sure that you are using the latest codes and checkpoints, especially if you used Qwen-7B before 09.25.2023.请使用最新模型和代码，尤其如果你在9月25日前已经开始使用Qwen-7B，千万注意不要使用错误代码和模型。
# Try importing flash-attention for faster inference..."""

# inference with model.generate
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    do_sample=True,
    top_k=10,
    temperature=0.01,
    min_new_tokens=1,
    eos_token_id=[151643],
    forced_eos_token_id=[151643, 26840, 16660],
)[0]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [11]:
print(tokenizer.decode(outputs, skip_special_tokens=False))

###
Article: (CNN Student News) -- November 9, 2012 . Download PDF maps related to today's show: . Greece . Guatemala . Japan . Michigan; Utah . Click here to access the transcript of today's CNN Student News program. Please note that there may be a delay between the time when the video is available and when the transcript is published.

Summarize the above article in 3 sentences.
The daily transcript is a written version of each day's CNN Student News program . Use this transcript to help students with reading comprehension and vocabulary . Use the weekly Newsquiz to test your knowledge of stories you saw on CNN Student News .

###
Article: KATHMANDU, Nepal (CNN) -- Two people were killed and about a dozen others were injured when a bomb exploded in a Catholic church in Kathmandu on Saturday morning, police said. The damage inside the church in Kathmandu following Saturday's bomb blast. The explosion in the Nepalese capital killed a 15-year-old girl and a 30-year-old woman. "The bomb 